In [1]:
%load_ext autoreload
%autoreload 2

import http_request
import utils

In [2]:
filepath = '/home/gqy/Desktop/Http_Gengerator/Version_2/'

In [3]:
grammar_filename = 'grammar.conf'
grammar = utils.load(filepath+grammar_filename)
grammar

{'End_Line': ['\r\n'],
 'Header_Line_CRLF': ['\r\n'],
 'Request_Line': {'HTTP_Version': ['HTTP/1.1'],
  'Line_CRLF': ['\r\n'],
  'Method': ['GET'],
  'Request_URI': ['http://www.benigh.com/', 'http://www.attack.com/'],
  'Space': [' ']}}

In [4]:
rule_filename = 'rule.conf'
rule = utils.load(filepath+rule_filename)
rule

{'Header_Copy': [{'name': 'Host',
   'num': 2,
   'value': ['www.benigh.com', 'www.attack.com']},
  {'name': 'Connection', 'num': 1, 'value': ['keep-alive']}]}

In [5]:
request = http_request.Request()
print (request.__dict__)

{'Header_Lines': [], 'Headers': {}, 'End_Line': [], 'Header_Line_CRLF': [], 'Request_Line': []}


In [7]:
def conf_parse(conf_file,request):
    '''
    解析带有配置选项的文件conf_file,并给request的各属性赋值,如:
    * request.Headers
    * request.Request_Line
    * request.Line_CRLF
    * request.End_Line
    '''
    
    conf_dict = utils.load(conf_file)
    '''
    conf_dict中每个key对应的value的type有三种取值情况:
    * value是字符串列表，如 "Header_Line_CRLF": ["\r\n"]
    * value是dict
    * value是dict的列表
    '''
    for key,values in conf_dict.items():
        func = getattr(request,key.lower())
        
        #key对应的值是dict
        if isinstance(values,dict):
            func(**values)
        #key对应的值是列表
        elif isinstance(values,list):
            for value in values:
                #列表中的元素是dict
                if isinstance(value,dict):
                    func(**value)
                #列表中的元素不是dict，可能是数字、str或list
                else:
                    #注意:这里不是程序解包，是位置参数
                    func(values)
                    break    

In [8]:
conf_parse(grammar_filename,request)
print (request.__dict__)

{'Header_Lines': [], 'Headers': {}, 'End_Line': ['\r\n'], 'Header_Line_CRLF': ['\r\n'], 'Request_Line': ['GET http://www.benigh.com/ HTTP/1.1\r\n', 'GET http://www.attack.com/ HTTP/1.1\r\n']}


In [9]:
conf_parse(rule_filename,request)
print (request.__dict__)

{'Header_Lines': [], 'Headers': {'Connection': [(('keep-alive',), (0, 0))], 'Host': [(('www.benigh.com', 'www.attack.com'), (0, 0)), (('www.attack.com', 'www.benigh.com'), (0, 0))]}, 'End_Line': ['\r\n'], 'Header_Line_CRLF': ['\r\n'], 'Request_Line': ['GET http://www.benigh.com/ HTTP/1.1\r\n', 'GET http://www.attack.com/ HTTP/1.1\r\n']}


In [10]:
#生成该request的所有字符串表示
requests = request.get_request()
for req in requests:
    print (req)

GET http://www.benigh.com/ HTTP/1.1
Connection:keep-alive
Host:www.benigh.com
Host:www.attack.com


GET http://www.benigh.com/ HTTP/1.1
Connection:keep-alive
Host:www.attack.com
Host:www.benigh.com


GET http://www.attack.com/ HTTP/1.1
Connection:keep-alive
Host:www.benigh.com
Host:www.attack.com


GET http://www.attack.com/ HTTP/1.1
Connection:keep-alive
Host:www.attack.com
Host:www.benigh.com


GET http://www.attack.com/ HTTP/1.1
Connection:keep-alive
Host:www.benigh.com
Host:www.attack.com


GET http://www.attack.com/ HTTP/1.1
Connection:keep-alive
Host:www.attack.com
Host:www.benigh.com


